# Nested SQL Queries: Relational Analysis


In [1]:
# Core setup
import sqlite3
import pandas as pd
import numpy as np

pd.set_option("display.width", 120)
pd.set_option("display.max_columns", 50)
np.random.seed(42)

def q(conn, sql, params=None):
    """Run a SQL query and return a DataFrame."""
    return pd.read_sql_query(sql, conn, params=params or {})

In [2]:
# Utility helpers for richer outputs
import matplotlib.pyplot as plt

def show_schema(conn):
    tables = q(conn, "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' ORDER BY name;")
    print("Tables:", ", ".join(tables["name"].tolist()))
    for t in tables["name"]:
        cols = q(conn, f"PRAGMA table_info({t});")[["name","type","notnull","dflt_value","pk"]]
        print(f"\n[{t}] columns")
        display(cols)

def row_counts(conn):
    tables = q(conn, "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' ORDER BY name;")
    out=[]
    for t in tables["name"]:
        n = q(conn, f"SELECT COUNT(*) AS n FROM {t};")["n"].iloc[0]
        out.append((t, int(n)))
    df = pd.DataFrame(out, columns=["table","rows"]).sort_values("rows", ascending=False)
    display(df)

def plot_hist(series, title, bins=25):
    plt.figure()
    plt.hist(series, bins=bins)
    plt.title(title)
    plt.xlabel(series.name if series.name else "value")
    plt.ylabel("count")
    plt.show()

In [3]:
conn = sqlite3.connect(":memory:")
conn.execute("PRAGMA foreign_keys = ON;")
conn.executescript("""
CREATE TABLE movies(movie_id INTEGER PRIMARY KEY, title TEXT, release_year INT, rating REAL);
CREATE TABLE actors(actor_id INTEGER PRIMARY KEY, full_name TEXT);
CREATE TABLE movie_cast(movie_id INT, actor_id INT, role_type TEXT, UNIQUE(movie_id, actor_id));
""")

In [4]:
np.random.seed(21)
n_movies=220
years=np.random.randint(1995, 2025, size=n_movies)
movies=pd.DataFrame({
    "movie_id": range(1, n_movies+1),
    "title": [f"Film {i:03d}" for i in range(1, n_movies+1)],
    "release_year": years,
    "rating": np.round(np.clip(np.random.normal(6.7,1.0,size=n_movies),1.0,9.8),2)
})
movies.to_sql("movies", conn, if_exists="append", index=False)

n_actors=140
actors=pd.DataFrame({"actor_id": range(1, n_actors+1), "full_name": [f"Actor {i:03d}" for i in range(1, n_actors+1)]})
actors.to_sql("actors", conn, if_exists="append", index=False)

rows=[]
for mid in movies["movie_id"]:
    k=np.random.randint(3,9)
    chosen=np.random.choice(actors["actor_id"], size=k, replace=False)
    role_types=(["Lead"]*np.random.randint(1,3)+["Supporting"]*(k-1))[:k]
    np.random.shuffle(role_types)
    for aid, rt in zip(chosen, role_types):
        rows.append({"movie_id": int(mid), "actor_id": int(aid), "role_type": rt})
pd.DataFrame(rows).drop_duplicates(["movie_id","actor_id"]).to_sql("movie_cast", conn, if_exists="append", index=False)

1195

In [5]:
top_per_year = q(conn, """
SELECT m.release_year, m.title, m.rating
FROM movies m
WHERE m.rating = (SELECT MAX(m2.rating) FROM movies m2 WHERE m2.release_year = m.release_year)
ORDER BY m.release_year DESC
LIMIT 20;
""")
display(top_per_year)

,release_year,title,rating
0,2024,Film 145,8.26
1,2023,Film 155,7.81
2,2022,Film 119,8.43
3,2021,Film 058,9.79
4,2020,Film 096,7.72
5,2019,Film 192,8.57
6,2018,Film 141,8.05
7,2017,Film 121,7.77
8,2016,Film 105,9.27
9,2015,Film 057,8.40


## Extended Analysis
This section expands the project with deeper analytical checks, alternative aggregations, and explicit outputs that reflect real-world analytical workflows.

In [6]:
# Table-level row counts (defensive inspection)
tables = ["customers","orders","order_items","products","fact_sales","deliveries","movies","actors"]
print("Row counts by table:")
for tbl in tables:
    try:
        df_cnt = q(conn, "SELECT COUNT(*) AS n FROM " + tbl)
        print(f"{tbl}: {int(df_cnt['n'].iloc[0])}")
    except Exception:
        pass

Row counts by table:
movies: 220
actors: 140


In [7]:
# Category / segment distribution checks (when applicable)
try:
    dist = q(conn, 
        "SELECT category, COUNT(*) AS rows "
        "FROM order_items "
        "GROUP BY category "
        "ORDER BY rows DESC "
        "LIMIT 5"
    )
    print("Top categories by row count:")
    display(dist)
except Exception:
    pass

In [8]:
# Order value distribution summary
try:
    ov = q(conn,
        "SELECT "
        "ROUND(AVG(order_total),2) AS avg_order_value, "
        "ROUND(MAX(order_total),2) AS max_order_value "
        "FROM ("
        "   SELECT SUM(units * unit_price) AS order_total "
        "   FROM order_items "
        "   GROUP BY order_id"
        ")"
    )
    print("Order value distribution:")
    display(ov)
except Exception:
    pass

## Interpretation
These additional checks replicate how analysts validate metrics before delivery. Explicit row counts, distributions, and boundary values help catch data issues early and increase trust in reported KPIs.

## Top-N Within Group
Top-rated films per decade using window functions.

In [9]:
top_decade = q(conn, """
WITH m AS (
    SELECT movie_id, title, release_year, rating,
           (release_year / 10) * 10 AS decade
    FROM movies
),
ranked AS (
    SELECT decade, title, rating,
           ROW_NUMBER() OVER (PARTITION BY decade ORDER BY rating DESC, title ASC) AS rnk
    FROM m
)
SELECT decade, title, rating
FROM ranked
WHERE rnk <= 3
ORDER BY decade DESC, rnk ASC;
""")
display(top_decade)

,decade,title,rating
0,2020,Film 058,9.79
1,2020,Film 119,8.43
2,2020,Film 145,8.26
3,2010,Film 002,9.30
4,2010,Film 105,9.27
5,2010,Film 113,8.66
6,2000,Film 177,8.46
7,2000,Film 154,8.28
8,2000,Film 088,8.17
9,1990,Film 041,8.36


## Actor Network Signals
Broad reach and quality-weighted exposure.

In [10]:
actor_signal = q(conn, """
WITH actor_stats AS (
    SELECT mc.actor_id,
           COUNT(DISTINCT mc.movie_id) AS films,
           AVG(m.rating) AS avg_rating,
           SUM(CASE WHEN mc.role_type='Lead' THEN 1 ELSE 0 END) AS lead_films
    FROM movie_cast mc
    JOIN movies m ON m.movie_id = mc.movie_id
    GROUP BY mc.actor_id
)
SELECT a.full_name,
       films,
       lead_films,
       ROUND(avg_rating, 2) AS avg_rating,
       ROUND(films * avg_rating, 2) AS reach_score
FROM actor_stats s
JOIN actors a ON a.actor_id = s.actor_id
WHERE films >= 5
ORDER BY reach_score DESC
LIMIT 20;
""")
display(actor_signal)

,full_name,films,lead_films,avg_rating,reach_score
0,Actor 102,18,6,7.09,127.58
1,Actor 023,16,5,7.07,113.04
2,Actor 052,17,5,6.40,108.74
3,Actor 080,16,4,6.62,105.88
4,Actor 122,16,1,6.52,104.39
5,Actor 064,14,4,6.90,96.65
6,Actor 135,14,6,6.65,93.06
7,Actor 100,14,0,6.50,91.04
8,Actor 098,13,2,6.86,89.22
9,Actor 009,13,6,6.79,88.28
